In [12]:
import threading
import torch
import time

In [11]:
import dis
dis.dis("total += 1")

  1           0 LOAD_NAME                0 (total)
              2 LOAD_CONST               0 (1)
              4 INPLACE_ADD
              6 STORE_NAME               0 (total)
              8 LOAD_CONST               1 (None)
             10 RETURN_VALUE


In [13]:
# fine-grained locking (only hold the lock for a small piece of work)
lock = threading.Lock()
total = torch.tensor(0, dtype=torch.int32)

def inc(count):
    global total
    for i in range(count):
        lock.acquire()
        total += 1
        lock.release()

# inc(1000)
t1 = threading.Thread(target=inc, args=[1_000_000])
t2 = threading.Thread(target=inc, args=[1_000_000])

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()
print(end-start, "seconds")
total

29.221033811569214 seconds


tensor(2000000, dtype=torch.int32)

In [14]:
# coarse-grained locking (only hold the lock for a small piece of work)
lock = threading.Lock()
total = torch.tensor(0, dtype=torch.int32)

def inc(count):
    global total
    lock.acquire()
    for i in range(count):
        total += 1
    lock.release()

t1 = threading.Thread(target=inc, args=[1_000_000])
t2 = threading.Thread(target=inc, args=[1_000_000])

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()
print(end-start, "seconds")
total

10.285215377807617 seconds


tensor(2000000, dtype=torch.int32)

In [21]:
bank_accounts = {"x": 25, "y": 100, "z": 200} # in dollars
lock = threading.Lock() # protects bank_accounts

def transfer(src, dst, amount):
    with lock:
        #lock.acquire()
        success = False
        if bank_accounts[src] >= amount:
            bank_accounts[src] -= amount
            bank_accounts[dst] += amount
            success = True
        print("transferred" if success else "denied")
        # lock.release()    with statement calls automatically, even with exceptions

In [22]:
transfer("x", "y", 20)

transferred


In [23]:
transfer("x", "z", 10)

denied


In [24]:
# EXCEPTION!
transfer("w", "x", 50)

KeyError: 'w'

In [25]:
transfer("z", "x", 50)

transferred
